In [614]:
import pandas as pd

In [615]:
# Get the data

def getData():

	import requests

	files = [
	"time_series_19-covid-Confirmed.csv",
	"time_series_19-covid-Deaths.csv",
	"time_series_19-covid-Recovered.csv"
	]

	headers = {'Accept': 'application/vnd.github.v3.raw'}

	for path in files:
		url = "https://api.github.com/repos/CSSEGISandData/COVID-19/contents/csse_covid_19_data/csse_covid_19_time_series/{path}".format(path=path)
		print("Getting", path)
		r = requests.get(url, headers=headers)
		with open(path, 'w') as f:
			f.write(r.text)
			
	print("Files saved")

getData()

Getting time_series_19-covid-Confirmed.csv
Getting time_series_19-covid-Deaths.csv
Getting time_series_19-covid-Recovered.csv
Files saved


In [616]:
# Read in the data and have a look
confirmed = pd.read_csv("time_series_19-covid-Confirmed.csv")
confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20
0,NaN,Thailand,15.0000,101.0000,2,3,5,7,8,8,...,48,50,50,50,53,59,70,75,82,114
1,NaN,Japan,36.0000,138.0000,2,1,2,2,4,4,...,420,461,502,511,581,639,639,701,773,839
2,NaN,Singapore,1.2833,103.8333,0,1,3,3,4,5,...,130,138,150,150,160,178,178,200,212,226
3,NaN,Nepal,28.1667,84.2500,0,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,NaN,Malaysia,2.5000,112.5000,0,0,0,3,4,4,...,83,93,99,117,129,149,149,197,238,428


In [617]:
# Clear out regions and lat/lon / general data cleanup
colnames = list(confirmed.columns.values)
dates = colnames[4:]
melted_confirmed = confirmed.melt(id_vars='Country/Region', var_name='Date', value_name='ConfirmedCases')
melted_confirmed = melted_confirmed.groupby(['Date','Country/Region'])['ConfirmedCases'].sum().reset_index()
melted_confirmed = melted_confirmed = melted_confirmed[melted_confirmed['Date'].isin(dates)]
melted_confirmed = melted_confirmed.sort_values(by=['ConfirmedCases'])
melted_confirmed['Date'] = pd.to_datetime(melted_confirmed['Date'], infer_datetime_format=True)  
melted_confirmed['ConfirmedCases'] = melted_confirmed['ConfirmedCases'].astype(int)
melted_confirmed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7938 entries, 0 to 6643
Data columns (total 3 columns):
Date              7938 non-null datetime64[ns]
Country/Region    7938 non-null object
ConfirmedCases    7938 non-null int32
dtypes: datetime64[ns](1), int32(1), object(1)
memory usage: 217.1+ KB


In [618]:
# Read in population data, and only include countries with >100 cases
country_pop = pd.read_csv('Dev/country_population.csv')
melted_confirmed_wpop = pd.merge(melted_confirmed, country_pop, on = 'Country/Region', how = 'left').fillna(0)
melted_confirmed_wpop.rename(columns = {'Country/Region':'Country'}, inplace = True) 
melted_confirmed_wpop['Population'] = melted_confirmed_wpop['Population'].astype(int)
melted_confirmed_wpop['population_proportion'] = (melted_confirmed_wpop['ConfirmedCases'] / melted_confirmed_wpop['Population'])*100
melted_confirmed_wpop = melted_confirmed_wpop[melted_confirmed_wpop['ConfirmedCases'] > 100]
melted_confirmed_wpop['day_since_100'] = melted_confirmed_wpop.groupby('Country').cumcount() + 1
melted_confirmed_wpop['dod_growth'] = melted_confirmed_wpop['population_proportion'].pct_change()
melted_confirmed_wpop = melted_confirmed_wpop[melted_confirmed_wpop['Country'] != 'Cruise Ship']

In [622]:
# Look at the first week
first_week = days_since_dataframe[days_since_dataframe['day_since_100'] < 14] #change this to adjust timeframe
first_week = pd.DataFrame(first_week.pivot(index='Country', columns='day_since_100', values=['ConfirmedCases']).fillna("").reset_index())
first_week

Country ConfirmedCases                              \
day_since_100                              1    2    3     4     5     6   
0                   Australia          107.0  128  128   200   250         
1                     Austria          104.0  131  182   246   302   504   
2                     Bahrain          110.0  189  195   195   210         
3                     Belgium          109.0  169  200   239   267   314   
4                      Brazil          151.0  151                          
5                      Canada          108.0  117  193   198               
6                       China          548.0  643  920  1406  2075  2877   
7                     Czechia          141.0  189                          
8                     Denmark          264.0  444  617   804   836         
9                       Egypt          109.0                               
10                    Estonia          115.0                               
11                    Finland          155.0  225                          
12                     France          130.0  191  204   288   380   656   
13                    Germany          130.0  159  196   262   482   670   
14                     Greece          190.0  228                          
15                    Iceland          103.0  134  156                     
16                      India          102.0                               
17                       Iran          139.0  245  388   593   978  1501   
18                       Iraq          101.0  110                          
19                    Ireland          129.0                               
20                     Israel          109.0  131  161   193               
21                      Italy          155.0  229  322   453   655   888   
22                      Japan          105.0  122  147   159   170   189   
23               Korea, South          104.0  204  433   602   833   977   
24                     Kuwait          104.0                               
25                   Malaysia          117.0  129  149   149   197   238   
26                Netherlands          128.0  188  265   321   382   503   
27                     Norway          108.0  147  176   205   400   598   
28                Philippines          111.0                               
29                     Poland          103.0                               
30                   Portugal          112.0  169                          
31                      Qatar          262.0  262  320   337               
32                    Romania          123.0                               
33               Saudi Arabia          103.0                               
34                  Singapore          102.0  106  108   110   110   117   
35                   Slovenia          141.0  181                          
36                      Spain          120.0  165  222   259   400   500   
37                     Sweden          101.0  161  203   248   355   500   
38                Switzerland          114.0  214  268   337   374   491   
39                         US          118.0  149  217   262   402   518   
40             United Kingdom          116.0  164  207   274   322   384   

                                                            
day_since_100     7     8     9    10     11     12     13  
0                                                           
1               655                                         
2                                                           
3               314   559   689                             
4                                                           
5                                                           
6              5509  6087  8141  9802  11891  16630  19716  
7                                                           
8                                                           
9                                                           
10                   

In [620]:
#Look at a few countries
melted_confirmed_wpop = melted_confirmed_wpop[(melted_confirmed_wpop['Country'] == 'Austria') | (melted_confirmed_wpop['Country'] == 'Australia') | (melted_confirmed_wpop['Country'] == 'Belgium') | (melted_confirmed_wpop['Country'] == 'Japan')]
melted_confirmed_wpop.groupby(['Continent','Country']).count()

Date  ConfirmedCases  Population  population_proportion  \
Continent Country                                                              
Asia      Japan        24              24          24                     24   
Europe    Austria       8               8           8                      8   
          Belgium      10              10          10                     10   
Oceania   Australia     6               6           6                      6   

                     day_since_100  dod_growth  
Continent Country                               
Asia      Japan                 24          24  
Europe    Austria                8           8  
          Belgium               10          10  
Oceania   Australia              6           6

In [621]:
#Plot it

import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML
fig = px.line(melted_confirmed_wpop[melted_confirmed_wpop['day_since_100'] > 1], x="day_since_100", y="ConfirmedCases", color='Country', line_group="Country")
fig.show() 